In [167]:
from plotly import __version__
from plotly import offline as pyoff

In [168]:
pyoff.init_notebook_mode(connected=True)

In [230]:
import pandas as pd
import numpy as np
import gensim
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import logging
import multiprocessing
import os
from collections import namedtuple

FORMAT = '%(asctime)s %(levelname)s %(message)s'
DATEFORMAT = '%Y-%m-%d %H:%M:%S'
logging.basicConfig(level=logging.INFO,
                    format=FORMAT,
                    datefmt=DATEFORMAT)

In [170]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir = os.path.join(parent_dir, 'data')
models_dir = os.path.join(parent_dir, 'models')
print('working directory: ', os.getcwd())
print('data directory:    ', data_dir, )
print('models directory:  ', models_dir)

working directory:  /Volumes/Datahouse/Users/Stipe/Documents/Studium/Master VWL/5 WS 2017/Seminar Information Systems/InformationSystemsWS1718/notebooks
data directory:     /Volumes/Datahouse/Users/Stipe/Documents/Studium/Master VWL/5 WS 2017/Seminar Information Systems/InformationSystemsWS1718/data
models directory:   /Volumes/Datahouse/Users/Stipe/Documents/Studium/Master VWL/5 WS 2017/Seminar Information Systems/InformationSystemsWS1718/models


In [171]:
data = pd.read_pickle(os.path.join(data_dir, 'data_clean_4cols.pickle'))
data.sample(n=20)

,id,from_name,message,Partei_ABK
155013,702563316515363_1116179371820420,Michael Thews - MdB,Herzlichen Glückwunsch Frank-Walter!,SPD
36794,183467831832037_736297576549057,Ursula Groden-Kranich,Finale der Fernsehsitzung,CDU
32092,93711056469_10154095213831470,Kai Gehring,"Grund zur Freude: Heute Eröffnung des ""Interdi...",GRÜNE
15195,1847338492218497_1909138356038510,Dr. Carsten Brodesser,Meine persönliche Empfehlung für alle Eifelfans:,CDU
142612,356722457844832_778884898961917,Rita Schwarzelühr-Sutter,Interessanter Besuch bei Franke Foodservice Sy...,SPD
176587,21289227249_10155176031077250,FDP Party,Das Programm von Martin Schulz könnte tödlich ...,FDP
47537,475865579194090_1212190232228284,Marcus Held,Eine tolle Aktion des Eleonoren-Gymnasium Worm...,SPD
20021,1169629846446002_1438220509586933,Jan Deboy Bundestagskandidat,Wertschätzung für das FriseurInnen-Handwerk in...,SPD
92133,647898078564416_1509535845733964,Till Mansmann,Bei Wünschdirwas immer ganz oben: der Weltfrie...,FDP
41630,385433161521193_1408979702499862,Ulrich Hampel,Heute blicken wir in der NRW-Landesgruppe in d...,SPD


In [290]:
candidate_data = (data.drop(['id', 'message'], axis=1)
                      .drop_duplicates('from_name')
#                      .set_index('from_name')
                 )

candidate_data.sample(n=10)

,from_name,Partei_ABK
117577,Dr. Sascha Raabe,SPD
81334,Silke Leber,FDP
72837,Carina Konrad,FDP
19776,Britta Dassler,FDP
171421,Paul Ziemiak,CDU
152482,Thomas Stritzl MdB,CDU
135955,Joachim Schneider,AfD
139571,Martin Schulz,SPD
15368,Karl-Heinz Brunner,SPD
147994,Albert Stegemann,CDU


In [173]:
# removes party mentions to test w
for char in ['SPD', 'spd', 'FDP', 'fdp', 'CDU' 'cdu', 'AfD' 'afd', 'AFD', 'Grüne', 'GRÜNE', 'Die Grünen', 'GRÜNEN', 'Linke', 'LINKE', 'CSU', 'csu', 'Die Linke', 'DIE LINKE',]:
        data.message = data.message.str.replace(char, '')

In [174]:
any(data.message.str.count('FDP')>0)

False

In [244]:
from nltk.tokenize.casual import TweetTokenizer

tokenizer = TweetTokenizer
stopword_set = set(stopwords.words('german'))
MessageDoc = namedtuple('MessageDoc', 'words tags split')
alldocs = []  # Will hold all doacs in original order
for line_no, line in data.iterrows():
    #import pdb; pdb.set_trace()
    message = line.message.lower()
    words = for word in tokenizer().tokenize(message) if word not in stopword_set
    tags = [str(line_no), line['from_name']] #, line['Partei_ABK']] # line_no needs to be converted as string to be included in tags 
    split = ['train', 'test', 'extra', 'extra'][line_no//200000]  # 25k train, 25k test, 25k extra
    alldocs.append(MessageDoc(words, tags, split))

In [245]:
tags

['177306', 'DIE LINKE Party']

In [246]:
len(alldocs)

177307

In [247]:
cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

In [248]:
alldocs[0:9]

[MessageDoc(words=['gerade', 'einmal', '9.000', 'anträge', 'auf', 'die', 'kaufprämie', 'für', '#elektroautos', 'in', 'sechs', 'monaten', '–', 'weniger', 'als', 'die', 'hälfte', 'davon', 'durch', 'privatpersonen', '!', 'aber', 'eben', 'auch', 'kein', 'wunder', ',', 'wenn', 'sich', 'der', 'staat', 'auf', 'blindes', 'subventionieren', 'beschränkt', ',', 'anstatt', 'die', 'erforschung', 'von', 'speichermöglichkeiten', 'zu', 'fördern', 'oder', 'die', 'ladeinfrastruktur', 'auszubauen', '…', '#', '#btw17', '#hohenlohe', '#schwäbischhall', '#umwelt'], tags=['0', 'Valentin Abel'], split='train'),
 MessageDoc(words=['"', 'die', 'liberalen', 'stehen', 'für', 'einen', 'individualistischen', 'freiheitsbegriff', ',', 'für', 'himmelweite', 'bürgerrechte', '.', 'zuversicht', 'und', 'hoffnung', 'sind', 'ihnen', 'näher', 'als', 'wut-und-blut-reden', '.', 'sie', 'durchdenken', 'die', 'digitale', 'zukunft', ',', 'widmen', 'sich', 'der', 'bildungspolitik', ',', 'suchen', 'die', 'diskussion', 'über', 'wirts

In [304]:
from collections import OrderedDict

models = [
    gensim.models.Doc2Vec(dm=0, size=60, negative=5, hs=0, min_count=5, iter=5, workers=cores),
    gensim.models.Doc2Vec(dm=0, size=100, negative=5, hs=0, min_count=5, iter=5, workers=cores),
    gensim.models.Doc2Vec(dm=1, dm_concat=1, size=100, window=5, negative=5, hs=0, min_count=5, iter=5, workers=cores),
    gensim.models.Doc2Vec(dm=1, dm_mean=1, size=100, window=10, negative=5, hs=0, min_count=5, iter=5, workers=cores),
]

In [305]:
# Speed up setup by sharing results of the 1st model's vocabulary scan
models[0].build_vocab(alldocs)  # PV-DM w/ concat requires one special NULL word so it serves as template
print(models[0])
for model in models[1:]:
    model.reset_from(models[0])
    print(model)

Doc2Vec(dbow,d60,n5,mc5,s0.001,t4)
Doc2Vec(dbow,d100,n5,mc5,s0.001,t4)
Doc2Vec(dm/m,d100,n5,w10,mc5,s0.001,t4)


In [306]:
models_by_name = OrderedDict((str(model), model) for model in models[0:])

from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
#models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([models[1], models[2]])
#models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([models[1], models[2]])

In [307]:
models_by_name

OrderedDict([('Doc2Vec(dbow,d60,n5,mc5,s0.001,t4)',
              <gensim.models.doc2vec.Doc2Vec at 0x1b91abf048>),
             ('Doc2Vec(dbow,d100,n5,mc5,s0.001,t4)',
              <gensim.models.doc2vec.Doc2Vec at 0x1b91abf198>),
             ('Doc2Vec(dm/m,d100,n5,w10,mc5,s0.001,t4)',
              <gensim.models.doc2vec.Doc2Vec at 0x1b91abf240>)])

In [308]:
for name, train_model in models_by_name.items():
    train_model.train(alldocs, total_examples=len(alldocs), epochs=5, start_alpha=0.025, end_alpha=0.001)

In [281]:
# saving the model doesn't work. Why?
#model.save(os.path.join(models_dir, 'doc2vec_c.model'))
#logger.info('model saved')

In [282]:
#doc_vecs.doctags['FDP']
#word_vecs.vocab

In [283]:
candidate_data.tail(10)

,from_name,Partei_ABK
173042,Dr. Daniela De Ridder,SPD
173559,Björn Simon,CDU
173902,Waldemar Westermayer,CDU
173963,AfD Party,AfD
174481,CDU Party,CDU
174974,SPD Party,SPD
175488,CSU Party,CSU
176078,GRÜNE Party,GRÜNE
176425,FDP Party,FDP
176969,DIE LINKE Party,DIE LINKE


In [291]:
# calculate similarity for all candidates and parties
for party in ['SPD Party', 'CDU Party', 'DIE LINKE Party', 'AfD Party', 'CSU Party', 'GRÜNE Party', 'FDP Party']:
    candidate_data[party] = candidate_data['from_name'].map(lambda candidate: models_by_name['Doc2Vec(dbow,d60,n5,mc5,s0.001,t4)'].docvecs.similarity(candidate, party))

# make a new column holding which party is most similar
candidate_data['most similar'] = candidate_data.loc[:,'SPD Party':].idxmax(axis=1)
candidate_data.head()

,from_name,Partei_ABK,SPD Party,CDU Party,DIE LINKE Party,AfD Party,CSU Party,GRÜNE Party,FDP Party,most similar
0,Valentin Abel,FDP,0.141927,0.131706,0.116456,0.056901,0.287501,0.181639,0.503164,FDP Party
93,Dr. Michael von Abercron,CDU,0.202028,0.341586,0.177481,0.007636,0.068244,0.175708,0.288894,CDU Party
168,Grigorios Aggelidis,FDP,0.310661,0.223307,0.075479,0.138598,0.168026,0.296508,0.194489,SPD Party
215,Diyar Agu,DIE LINKE,0.211816,0.351690,0.540049,0.020187,0.159524,0.284911,0.299974,DIE LINKE Party
269,Gökay Akbulut DIE LINKE,DIE LINKE,0.320121,0.198477,0.535448,0.142786,0.275604,0.398215,0.260205,DIE LINKE Party


In [292]:
candidate_data.shape

(1008, 10)

In [293]:
most_similar_candidates = pd.crosstab(candidate_data['Partei_ABK'], candidate_data['most similar'])

In [294]:
most_similar_candidates

most similar,AfD Party,CDU Party,CSU Party,DIE LINKE Party,FDP Party,GRÜNE Party,SPD Party
Partei_ABK,,,,,,,
AfD,103,0,5,7,3,4,2
CDU,13,133,31,3,12,9,8
CSU,0,0,46,0,0,0,0
DIE LINKE,1,1,1,107,0,1,0
FDP,9,3,18,4,128,6,5
GRÜNE,1,0,3,5,1,85,0
SPD,13,12,32,27,4,25,137


In [295]:
#sums = pd.crosstab(candidate_data['Partei_ABK'], candidate_data['most similar'])

In [296]:
model.docvecs.count

178315

In [354]:
party_colors = {'AfD': 'rgb(0, 0, 153)',
                'DIE LINKE': 'rgb(204, 0, 102)',
                'GRÜNE': 'rgb(0, 153, 0)',
                'CSU': 'rgb(102, 178, 255)',
                'CDU': 'rgb(0, 0, 0)',
                'FDP': 'rgb(255, 255, 51)',
                'SPD': 'rgb(255, 0, 0)'}
candidate_data['color'] = candidate_data['Partei_ABK'].map(party_colors)

leaders = ['Sahra Wagenknecht',
'Dietmar Bartsch',
'Katrin Göring-Eckardt',
'Cem Özdemir',
'Martin Schulz',
'Angela Merkel',
'Joachim Herrmann',
'Alexander Gauland',
'Alice Weidel']

candidate_data['leader'] = data['from_name'].isin(leaders)

candidate_data['size'] = 5
candidate_data.loc[candidate_data['from_name'].str.contains('Party'),'size'] = 10
candidate_data.loc[candidate_data['leader'], 'size'] = 8

candidate_data['symbol'] = 100
candidate_data.loc[candidate_data['from_name'].str.contains('Party'),'symbol'] = 101
candidate_data.loc[candidate_data[candidate_data['leader'], 'symbol'] = 102

In [356]:
candidate_data.tail(10)

6398      102
35073     102
49844     102
95280     102
108602    102
139571    102
162498    102
164334    102
Name: symbol, dtype: int64

In [37]:
from sklearn.manifold import TSNE
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

from time import time
import numpy as np
from sklearn import manifold
from sklearn.utils import check_random_state

In [357]:
import math

n_subplots = len(models_by_name)

plot_titles = list(models_by_name.keys())

fig2 = tools.make_subplots(rows=math.ceil(n_subplots/2), cols=2, subplot_titles=plot_titles)

row = 1
col = 1

count = 1

for name, trained_model in models_by_name.items():
    mask = [tag in candidate_data['from_name'].values for tag in trained_model.docvecs.offset2doctag]
    candidate_vecs = trained_model.docvecs.doctag_syn0[mask]
    
    X = candidate_vecs
    tsne = TSNE(n_components=2)
    X_tsne = tsne.fit_transform(X)
    
    trace = go.Scatter(x=X_tsne[:, 0], y=X_tsne[:, 1],
                       mode='markers',
                       marker=dict(color=candidate_data['color'],
                                   size=candidate_data['size'],
                                   symbol=candidate_data['symbol'],
                                   showscale=False,
                                   line=dict(color='black', 
                                             width=1)),
                       text=candidate_data['from_name'])
    
    fig2.append_trace(trace, row, col)
             
    if count % 2 == 0:
        row = row + 1
        col = 1
    else:
        col = col +1
        
    count += 1
    
fig2['layout'].update(title='Plots', showlegend=False, autosize=False, height=800, width=1000)
    
pyoff.iplot(fig2, filename='tsnePlot2')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [286]:
n_subplots = len(models_by_name)

cols = 2
d_array = np.array([{'is_3d': True}, {'is_3d': True}])
np.tile(d_array, (n_subplots/2, 1))

d3_array = np.arry([{'is_3d': True}])
np.repeat(3d_array, n_subplots)

fig3d = tools.make_subplots(rows=n_subplots//2, cols=2, 
                            specs=[[{'is_3d': True}, {'is_3d': True}]])

row = 1
col = 1

count = 1

for name, trained_model in models_by_name.items():
    mask = [tag in candidate_data['from_name'].values for tag in trained_model.docvecs.offset2doctag]
    candidate_vecs = trained_model.docvecs.doctag_syn0[mask]
    
    X = candidate_vecs
    tsne = TSNE(n_components=3)
    X_tsne = tsne.fit_transform(X)
    
    trace = go.Scatter3d(x=X_tsne[:, 0], y=X_tsne[:, 1], z=X_tsne[:, 2],
                       name=name,
                       mode='markers',
                       marker=dict(color=candidate_data['color'], 
                                   showscale=False,
                                   line=dict(color='black', width=1)),
                       text=candidate_data['from_name'])
    
    fig3d.append_trace(trace, row, col)
             
    if count % 2 == 0:
        row = row + 1
        col = 1
    else:
        col = col +1
        
    count += 1
    
fig3d['layout'].update(title='Plots', showlegend=False)
    
pyoff.iplot(fig3d, filename='tsnePlot2')

SyntaxError: invalid syntax (<ipython-input-286-b01d156a361a>, line 8)

In [212]:
import string
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [269]:
models_by_name[list('Doc2Vec(dbow,d60,n5,mc5,s0.001,t4)', 'dbow+dmc')]

TypeError: list() takes at most 1 argument (2 given)

In [331]:
models_by_name

OrderedDict([('Doc2Vec(dbow,d60,n5,mc5,s0.001,t4)',
              <gensim.models.doc2vec.Doc2Vec at 0x1b91abf048>),
             ('Doc2Vec(dbow,d100,n5,mc5,s0.001,t4)',
              <gensim.models.doc2vec.Doc2Vec at 0x1b91abf198>),
             ('Doc2Vec(dm/m,d100,n5,w10,mc5,s0.001,t4)',
              <gensim.models.doc2vec.Doc2Vec at 0x1b91abf240>)])

In [ ]:
models_by_name['Doc2Vec(dbow,d60,n5,mc5,s0.001,t4)'].corpus_count()

In [330]:
mask = [tag in candidate_data['from_name'].values for tag in models_by_name['dbow+dmm'].docvecs.offset2doctag]

KeyError: 'dbow+dmm'

In [332]:
alldocs

[MessageDoc(words=['gerade', 'einmal', '9.000', 'anträge', 'auf', 'die', 'kaufprämie', 'für', '#elektroautos', 'in', 'sechs', 'monaten', '–', 'weniger', 'als', 'die', 'hälfte', 'davon', 'durch', 'privatpersonen', '!', 'aber', 'eben', 'auch', 'kein', 'wunder', ',', 'wenn', 'sich', 'der', 'staat', 'auf', 'blindes', 'subventionieren', 'beschränkt', ',', 'anstatt', 'die', 'erforschung', 'von', 'speichermöglichkeiten', 'zu', 'fördern', 'oder', 'die', 'ladeinfrastruktur', 'auszubauen', '…', '#', '#btw17', '#hohenlohe', '#schwäbischhall', '#umwelt'], tags=['0', 'Valentin Abel'], split='train'),
 MessageDoc(words=['"', 'die', 'liberalen', 'stehen', 'für', 'einen', 'individualistischen', 'freiheitsbegriff', ',', 'für', 'himmelweite', 'bürgerrechte', '.', 'zuversicht', 'und', 'hoffnung', 'sind', 'ihnen', 'näher', 'als', 'wut-und-blut-reden', '.', 'sie', 'durchdenken', 'die', 'digitale', 'zukunft', ',', 'widmen', 'sich', 'der', 'bildungspolitik', ',', 'suchen', 'die', 'diskussion', 'über', 'wirts

In [334]:
any(a.words=='\x00' for a in alldocs)

False